### Test EpiFrag

In [1]:
#from configs.project_configs import GENOME_REFERENCE_FASTA as REF

REF = "/Users/billiegould/src/reference/"

#from core.genomic_utilities.methyl.epifrag import MethylAlignmentFile, ConsensusMethylatedFragment, MethylStatus
import sys
sys.path.append("/home/billiegould/src/Methyl/noise_motif_finder/")
from bg_epifrag import MethylatedFragment, MethylAlignmentFile, ConsensusMethylatedFragment, MethylStatus


from collections import defaultdict, Counter
from contextlib import closing
import pyfaidx
import pandas as pd
import numpy as np

print("A")




A


In [2]:
#test_region = ("chr1", 632108, 632228) # first region of V1 panel
test_region = ("chr1", 1206230, 1206960) # region 17

# a random bam from same fc as 0% methylated V1 sample
test_bam = "/home/billiegould/src/Methyl/noise_motif_finder/S_45_I_59-59_TTCGCCAT-TTCGCCAT_S45.marked.bam"
n_reads = 50

In [3]:
with closing(pyfaidx.Fasta(REF)) as ref:
    with MethylAlignmentFile(
            alignment_file=test_bam,
            ref=ref,
            ignore_last_n_bp_fragment=0,
            skip_duplicate_removal=False,
            min_mapq=50,
            max_unconverted_chh_fraction=0.15,
            fetch_consensus=True,
    ) as methyl_file:

        chrom, start, end = test_region

        # one dict for the region region
        region_motif_counts = {}

        n = 0
        for methyl_frag in methyl_file.fetch(contig=chrom, start=start, stop=end):
            # methyl_frag is ConsensusMethylatedFragment class, the ref seq between the two r1r2 read ends if reads pass filter
            if n == 45:
                fragment = methyl_frag.support_fragments[0]  # list of fragments that go into the consensus call
                print(f"FRAG: {fragment}")
                #x = fragment.test()
                #x = fragment.pass_chh_filter(3)
                #print(x)
                cpg_dict = fragment.get_methyl_status_loc_map().items()
                print(f"LOC MAP: {len(pd.DataFrame(cpg_dict))}")

                #cpg_dict_filt = fragment.get_methyl_status_loc_map_filtered().items()
                #print(f"NEW LOC MAP {len(pd.DataFrame(cpg_dict_filt))}")
                print(f"\n\n")
            n += 1
            if n >= n_reads:
                break

FastaNotFoundError: Cannot read FASTA file /static/bioinformatics-static-v20180124/references/genomes/hs38DH/hs38DH.fa

# make a feature to run this 

In [4]:
# create column metadata for feature
from core.gcloud.file import opener

In [7]:
# read in regions of interest

#434 modeling regions
with opener('gs://reference-datafiles/beds/freenome_v1_panel_manual_and_automated_per_region_hmf_definition_20190823.bed',"r") as handle:
    df_regions = pd.read_csv(handle, sep="\t", header=None)
    df_regions.columns = ['chrom', 'start', 'stop']
#     for _, row in df_regions.iterrows():
#         reg_name = str(row.chrom) + "_"+ str(row.start) + "_"+ str(row.stop)
#         modeling_regions[row['chrom']].append((row['start'],row['stop'], reg_name))
#         modeling_regions_names.append(reg_name)
df_regions.head()
df_regions.shape


(434, 3)

In [8]:
# create bed file for regions of interest
def make_metadata(df_regions, out_pfx:str):
    list_of_region_dicts = []
    for i, row in df_regions.iterrows():
        chrm, start, stop = row[0], row[1], row[2]
        #chrm, start, stop, genes = region
        list_of_region_dicts.append( {"chrm":chrm,
                                      "start":start,
                                      "stop":stop,
                                      #"cluster":cluster
                                     })

    thresh_dicts = [{f"thresh{n}": n} for n in [3,12,25]]    
    calc_dicts = [ {"col1":"total_frags"}, {"col2":"methyl_frags"}]

    np.savez_compressed(f'./{out_pfx}_col_metadata.npz', np.array(list_of_region_dicts), np.array(thresh_dicts), np.array(calc_dicts))
    
make_metadata(df_regions, "HMFC_by_min_cpg_CpG_motif_filtered")

In [ ]:
#created feature : HMFC_by_min_methyl_cpgs_CpG_filtered dim:(434,3,2,)

In [11]:
# find a test sample datastet
from featureio import FeatureIO

In [12]:
with FeatureIO() as ftio:
    dataset_list = ftio.get_datasets_for_training_class('colorectal_healthy_tbs_strict_discovery1j_v1_panelofneg')
    print(dataset_list)

['SD-000048F9', 'SD-000048FE', 'SD-00004912', 'SD-0000491B', 'SD-0000491C', 'SD-00004923', 'SD-00004925', 'SD-00004927', 'SD-00004930', 'SD-0000493C', 'SD-0000493D', 'SD-00004942', 'SD-00004944', 'SD-00004945', 'SD-00004946', 'SD-0000494B', 'SD-00004978', 'SD-0000499D', 'SD-000049B9', 'SD-000049D5', 'SD-000049E7', 'SD-000049F9']


In [ ]:
# replace epifragpy with bg_epifrag.py in local repo

# run build signature to analyze single sample
'''
build_signature --func summarizer.methods.bamstat_runner.run \
	--region gs://reference-datafiles/beds/freenome_v1_panel_manual_and_automated_per_region_hmf_definition_20190823.bed \
	--statistic_kwargs "{'min_methyl_cpgs': [3,12,25]}” \
	--methyl_frag_statistic HMFC
	--input SD-000048F9
	--out_path HMFC_by_min_methyl_cpgs_CpG_filtered \ 
	--version 1
	--c 16
'''

In [ ]:
# Note : V2 feature is run with the original epifrag code.
# V3 feature is run with the new epifrag code.

In [16]:
from pineapple.contrib.components.data_factories.matrix_factory import MatrixFactory as mf

ModuleNotFoundError: No module named 'pineapple'

In [ ]:
mat


In [15]:
help(mf)

NameError: name 'mf' is not defined

In [34]:
class myclass():
    number : int
    number2 : int
    
#    def __init__(self, number, number2):
#        self.i = number
#        self.j = number2

In [35]:
M = myclass(6,10)
print(M.number)

TypeError: myclass() takes no arguments

In [33]:
class myotherclass(myclass):
    arg1: str
    arg2: str



In [22]:
M = myotherclass(6)
print(M.i)

6


In [32]:
A = ""

if A:
    print("YAY")
else:
    print("Nope")

Nope


In [37]:
seq = 'ACTGACTGACTG'
print(seq[40:50])